In [ ]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
from datetime import datetime, timedelta

In [ ]:
df = pd.read_csv('/kaggle/input/stock-data/Nifty_50_with_indicators_.csv')
df['Date']=pd.to_datetime(df['Date'])
df = df.sort_values(by='Date')
df = df.set_index(pd.DatetimeIndex(df['Date'].values))
df=df[['Date', 'open', 'high','low', 'close']]['2004-07-02':'2005-07-02']
df

In [ ]:
# plotting close prices
fig = px.line(df, x='Date', y='close')
fig

In [ ]:
# calculate bollinger bands

# calculate sma
df['sma'] = df['close'].rolling(20).mean()

# calculate standard deviation
df['sd'] = df['close'].rolling(20).std()

# calculate lower band
df['lb'] = df['sma'] - 2 * df['sd']

# calculate upper band
df['ub'] = df['sma'] + 2 * df['sd']

df.dropna(inplace=True)
df

In [ ]:
# plotting close prices with bollinger bands
fig = px.line(df, x='Date', y=['close', 'sma', 'lb', 'ub'])
fig

In [ ]:
# find signals

def find_signal(close, lower_band, upper_band):
    if close < lower_band:
        return 'buy'
    elif close > upper_band:
        return 'sell'
    
    
df['signal'] = np.vectorize(find_signal)(df['close'], df['lb'], df['ub'])

df

In [ ]:
# creating backtest and position classes

class Position:
    def __init__(self, open_datetime, open_price, order_type, volume, sl, tp):
        self.open_datetime = open_datetime
        self.open_price = open_price
        self.order_type = order_type
        self.volume = volume
        self.sl = sl
        self.tp = tp
        self.close_datetime = None
        self.close_price = None
        self.profit = None
        self.status = 'open'
        
    def close_position(self, close_datetime, close_price):
        self.close_datetime = close_datetime
        self.close_price = close_price
        self.profit = (self.close_price - self.open_price) * self.volume if self.order_type == 'buy' \
                                                                        else (self.open_price - self.close_price) * self.volume
        self.status = 'closed'
        
    def _asdict(self):
        return {
            'open_datetime': self.open_datetime,
            'open_price': self.open_price,
            'order_type': self.order_type,
            'volume': self.volume,
            'sl': self.sl,
            'tp': self.tp,
            'close_datetime': self.close_datetime,
            'close_price': self.close_price,
            'profit': self.profit,
            'status': self.status,
        }
        
        
class Strategy:
    def __init__(self, df, starting_balance, volume):
        self.starting_balance = starting_balance
        self.volume = volume
        self.positions = []
        self.data = df
        
    def get_positions_df(self):
        df = pd.DataFrame([position._asdict() for position in self.positions])
        df['pnl'] = df['profit'].cumsum() + self.starting_balance
        return df
        
    def add_position(self, position):
        self.positions.append(position)
        
    def trading_allowed(self):
        for pos in self.positions:
            if pos.status == 'open':
                return False
        
        return True
        
    def run(self):
        for i, data in self.data.iterrows():
            
            if data.signal == 'buy' and self.trading_allowed():
                sl = data.close - 3 * data.sd
                tp = data.close + 2 * data.sd
                self.add_position(Position(data.Date, data.close, data.signal, self.volume, sl, tp))
                
            elif data.signal == 'sell' and self.trading_allowed():
                sl = data.close + 3 * data.sd
                tp = data.close - 2 * data.sd
                self.add_position(Position(data.Date, data.close, data.signal, self.volume, sl, tp))
                
            for pos in self.positions:
                if pos.status == 'open':
                    if (pos.sl >= data.close and pos.order_type == 'buy'):
                        pos.close_position(data.Date, pos.sl)
                    elif (pos.sl <= data.close and pos.order_type == 'sell'):
                        pos.close_position(data.Date, pos.sl)
                    elif (pos.tp <= data.close and pos.order_type == 'buy'):
                        pos.close_position(data.Date, pos.tp)
                    elif (pos.tp >= data.close and pos.order_type == 'sell'):
                        pos.close_position(data.Date, pos.tp)
                        
        return self.get_positions_df()



In [ ]:
# run the backtest
bollinger_strategy = Strategy(df, 10000, 100000)
result = bollinger_strategy.run()

result

In [ ]:
# plotting close prices with bollinger bands
fig = px.line(df, x='Date', y=['close', 'sma', 'lb', 'ub'])

# adding trades to plots
for i, position in result.iterrows():
    if position.status == 'closed':
        fig.add_shape(type="line",
            x0=position.open_datetime, y0=position.open_price, x1=position.close_datetime, y1=position.close_price,
            line=dict(
                color="green" if position.profit >= 0 else "red",
                width=3)
            )
fig

In [ ]:
# visualizing the results of the backtest
px.line(result, x='close_datetime', y='pnl')